<div>
    <h1><center>
        <img src="https://its-live-data.s3.amazonaws.com/documentation/ITS_LIVE_logo.png" width="500"/>
        </center></h1>
    <h1><center>
        Global glacier velocity point data access<br>
        using an ipyLeaflet basemap<br>
        </center></h1>
</div>

***

Author: Mark Fahnestock, Geophysical Institute, University of Alaska Fairbanks
Date: November 8, 2021

***

# Overview

This notebook allows you to select a set of point locations on a Leaflet-based global map, fetch all available ITS_LIVE glacier velocities for the locations, and plot the results.

Select the points by double-clicking on the map - click and drag on the map to pan the field of view.

Once points are chosen, hit "Make Plot" (bottom right corner) to produce a plot.

You can drag individual points after they are placed to relocate them, and then "Make Plot" again

Click "Clear Points" to start over.

The notebook is set up to show data coverage for a small set of "All Satellite" datacubes by default (parts of Greenland/Svalbard/Alaska only for now), but if you select Landsat 8 it will instead use a global collection of Landsat 8 datacubes.

Underling data is stored on AWS S3 as Zarr datacubes and is accessed without an intermediate server. Glacier velocities in the "All Satellite" datacube collection are derived from all available Landsat 8, Sentinel-1A/B, Sentinel-2A/B imagery, while they are limited to Landsat 8 for now in the global dataset.


Please refer to the <a href="https://its-live.jpl.nasa.gov/">project website</a> for further product infomation and for appraopriate data citation.

### Setting up a local environment

From the repository root, run in a terminal:
```shell
conda env create -f binder/environment.yml
```
activate newly created environment:
```shell
conda activate itslive-notebooks
```

then start jupyter in browser
```shell
cd notebooks
jupyter notebook
```

In [ ]:
%matplotlib widget
# import the library
from velocity_widget import ITSLIVE

velocity_widget = ITSLIVE()

In [ ]:
# If this congiguration changes we need to rerun the cell.
config = { 
    "plot": "v", # or other ITS_LIVE variables: vx, vy ...
    "min_separation_days": 1,
    "max_separation_days": 90,
    "color_by": "points" # valid values: satellite, points
}

velocity_widget.set_config(config)
velocity_widget.display()

## Gathering and processing the data

## Zarr and S3 cloud storage

* Why Zarr? 
* Chunking of ITS_LIVE cubes

### Working with ITS_LIVE zarr cubes in **xarray**

In [ ]:
# we can work directly with the xarray instances for the cubes that match our points in the map.
# we can directly access a dictionary of currently open ITL_LIVE datacubes containing the points picked above
# the keys for this dictionary are the s3 URI's for the xarray stores in zarr format
zarr_cubes_dict = velocity_widget.dct.open_cubes
print(f"open datacubes (xarray datasets in zarr format): {zarr_cubes_dict.keys()}")


In [ ]:
# The actual xarray is returned when the dictionary is indexed by the URI key 
#  - when opened on S3 it will take ~5-10 seconds to load the first time 
# Chunking was optimized for time series extraction at any given x and y.

# NOTE that these cubes are open, but only the metadata and coordinates have been downloaded, not the data itself
# THE DATA VOLUME IN THESE CUBES IS QUITE LARGE, fully loading variables for a whole cube could extend to multiple GB

# here jupyter/ipython will pretty-print a table of metadata and variables for a cube if we reference one that is open:
first_key = list(zarr_cubes_dict.keys())[0]
print(f'First S3 URI for open cube: {first_key}')
cube = zarr_cubes_dict[first_key]
cube